In [ ]:
# !pip install pandas matplotlib scipy numpy qutip

In [1]:
import numpy as np
import scipy.special as sp

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import Kite_Transmon_datagen

from concurrent.futures import ProcessPoolExecutor

from tqdm import tqdm
 

In [3]:
def relaxation_rates(dataset, Q_diel=1e6, omega_diel=6*1e9, Q_ind=500e6, omega_ind= .5*1e9, M = 5e-9, T = 20e-3):
    h = 6.62607015e-34

    Kb = 1.380649e-23
    e = 1.602176634e-19

    state_pairs = [(0, 1), (1, 2), (2, 3), (0, 3)]

    for (i, j) in state_pairs:
        omega_ij = dataset[f'omega_{i}{j}']
        
        N_ij = dataset[f'N_{i}{j}']
        n_sigma_ij = dataset[f'n_sigma_{i}{j}']
        n_delta_ij = dataset[f'n_delta_{i}{j}']
        phi_sigma_ij = dataset[f'phi_sigma_{i}{j}']
        phi_delta_ij = dataset[f'phi_delta_{i}{j}'] 

        mat_elem_cap_JJ1 = np.abs(N_ij(i,j) - .5*(n_sigma_ij(i,j) + n_delta_ij(i,j)) )**2
        mat_elem_cap_JJ2 = np.abs(N_ij(i,j) - .5*(n_sigma_ij(i,j) - n_delta_ij(i,j)))**2
        mat_elem_cap_Cap = np.abs(N_ij(i,j))**2

        mat_elem_ind_arm1 = np.abs(.5*(phi_sigma_ij(i,j) + phi_delta_ij(i,j)))**2
        mat_elem_ind_arm2 = np.abs(.5*(phi_sigma_ij(i,j) - phi_delta_ij(i,j)))**2

        hwb = (h*omega_ij)/(Kb*T)

        dataset.loc[:,f'Q_cap_{i}{j}'] = Q_diel*(omega_diel/np.abs(omega_ij))**.7
        dataset.loc[:,f'S_cap_{i}{j}'] = 8*(dataset.loc[:,'E_CJ']*1e9)/ dataset.loc[:,f'Q_cap_{i}{j}'] * np.tanh(hwb/2 )**-1 # multiplied by 4e^2 where e = charge

        dataset.loc[:,f'Q_ind_func_{i}{j}'] = Q_ind * sp.kn(0,(h*omega_ind)/(2*Kb*T))*np.sinh((h*omega_ind)/(2*Kb*T))/(
        sp.kn(0,np.abs(hwb)/2)*np.sinh(np.abs(hwb)/2))
        dataset.loc[:,f'S_ind_{i}{j}'] = (dataset.loc[:,'E_L']*1e9)/ dataset.loc[:,f'Q_ind_func_{i}{j}'] *(np.tanh(hwb/2)**-1) # multiplied by (phi_0)^2


        dataset.loc[:,f'Gamma_cap_{i}{j}'] = (mat_elem_cap_JJ1 + mat_elem_cap_JJ2) *dataset.loc[:,f'S_cap_{i}{j}'] \
            + mat_elem_cap_Cap * dataset.loc[:,f'S_cap_{i}{j}'] /(dataset.loc[:,'E_CJ']*1e9)*(dataset.loc[:,'E_Cs']*1e9)
        
        dataset.loc[:,f'Gamma_ind_{i}{j}'] = (mat_elem_ind_arm1 + mat_elem_ind_arm2) * dataset.loc[:,f'S_ind_{i}{j}']

        dataset.loc[:,f'Gamma_Fluxline_{i}{j}'] = M**2 *(dataset.loc[:,'E_L']*1e9)*  np.abs(phi_delta_ij)**2 * omega_ij / 25 * (np.tanh(hwb/ 2)**-1)

        dataset.loc[:,f'T1_{i}{j}'] = 1/(dataset.loc[:,f'Gamma_cap_{i}{j}'] + dataset.loc[:,f'Gamma_ind_{i}{j}'])



def dephasing_rates(dataset, A_phi=2*np.pi*3e-6, T = 20e-3, T_cav=50e-3):
    h_over_kb= 6.62607015e-34 /1.380649e-23

    dphi_ext=1e-5*2*np.pi

    dataset.loc[:,'domega_dphi'] = (dataset.loc[:,'omega_01_dphip'] - dataset.loc[:,'omega_01_dphim'] ) / (2*dphi_ext)
    domega_dphi_2 = ( dataset.loc[:,'omega_01_dphip'] - 2*dataset.loc[:,'omega_01'] + dataset.loc[:,'omega_01_dphim'] ) / (dphi_ext**2)
    dataset.loc[:,'Gamma_flux'] = A_phi**2*np.abs(domega_dphi_2)
    
    dataset.loc[:,'Gamma_chg_drift'] = np.pi/(2*np.exp(1))**2*dataset.loc[:,'omega_01']
    
    chi = dataset.loc[:,'omega_23'] - dataset.loc[:,'omega_01']
    omega_th=(dataset.loc[:,'omega_03']+dataset.loc[:,'omega_12'])/2

    n_th=(np.exp(h_over_kb*(omega_th)/T) -1)**-1

    kappa=dataset.loc[:,'T1_03']**-1*np.tanh(.5*h_over_kb*(omega_th)/T)

    dataset.loc[:,'Gamma_th'] = (n_th*kappa* chi**2/(kappa**2+chi**2))

    # dataset.loc[:,'Gamma_th'] = 

    dataset.loc[:,'T2'] = 1/(dataset.loc[:,'Gamma_th']+dataset.loc[:,'Gamma_chg_drift']+dataset.loc[:,'Gamma_flux'])


In [4]:
def data_formatting(filename):
    data = pd.read_csv(filename)

    # Convert string columns to complex numbers
    data = data.applymap(lambda x: complex(x) if isinstance(x, str) and 'j' in x else x)

    # Convert all complex columns to their abs
    data = data.applymap(lambda x: np.abs(x) if isinstance(x, complex) else x)

    EL_sweep = np.unique(data.loc[:,'E_L'])
    EJ_sweep = np.unique(data.loc[:,'E_J'])
    ECj_sweep = np.unique(data.loc[:,'E_Cj'])
    ECs_sweep = np.unique(data.loc[:,'E_Cs'])

    data.loc[:,'omega_p'] = round(np.sqrt(8*data.loc[:,'E_J']*data.loc[:,'E_Cj']),8)
    data.loc[:,'EJ/ECj'] = round(data.loc[:,'E_J']/data.loc[:,'E_Cj'],8)
    data.loc[:,'EJ/EL'] = round(data.loc[:,'E_J']/data.loc[:,'E_L'],8)
    data.loc[:,'EL/ECj'] = round(data.loc[:,'E_L']/data.loc[:,'E_Cj'],8)
    data.loc[:,'EL/ECs'] = round(data.loc[:,'E_L']/data.loc[:,'E_Cs'],8)

    return data, EL_sweep, EJ_sweep, ECj_sweep, ECs_sweep

In [6]:
filename= '/workspace/data/data_simulation_EJoEC.csv'

all_params = Kite_Transmon_datagen.CSV_gen(EJ_EC_sweep_bool=False,EJ2_EL_sweep_bool = True)
if __name__ == '__main__':
    with ProcessPoolExecutor() as executor:
            results = list(tqdm(executor.map(Kite_Transmon_datagen.process, all_params), total=len(all_params)))

# Convert results to a DataFrame
res = pd.DataFrame(results)

compression_options = dict(method='zip', archive_name=f'{filename}.csv')
res.to_csv(f'{filename}.zip', compression=compression_options, index=False)

  0%|          | 0/33 [00:00<?, ?it/s]

In [3]:
dataset,_,_,_,_ = data_formatting(filename)
relaxation_rates(dataset)
dephasing_rates(dataset)

NameError: name 'filename' is not defined

In [2]:
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import LogFormatter, SymmetricalLogLocator, LogFormatterSciNotation

def data_3Dplot(data, var1, var2, var3):
    
    X,Y=np.meshgrid(np.unique(data[var1]),np.unique(data[var2]))
    Z=data.pivot_table(index=var1, columns=var2, values=var3).values

    ax = plt.figure().add_subplot(projection='3d')
    # Plot the 3D surface
    ax.plot_surface(X, Y, Z.T, edgecolor='royalblue', lw=0.5, alpha=0.5)
    # surf = ax.plot_surface(X, Y, Z.T, cmap='viridis')
    ax.set(xlabel=var1, ylabel=var2, zlabel=var3)

    # Plot projections of the contours for each dimension.  
    ax.contour(X, Y, Z.T, zdir='z', offset=np.min(Z), cmap='coolwarm')
    ax.contour(X, Y, Z.T, zdir='x', offset=np.min(X), cmap='coolwarm')
    ax.contour(X, Y, Z.T, zdir='y', offset=np.max(Y), cmap='coolwarm')
    plt.show()
    return ax

def plot_2D(data, X_label, Y_label, Z_label, log_scale_xy = True, log_scale_z = True):
    
    X,Y = np.meshgrid(np.unique(data[X_label]),np.unique(data[Y_label]))
    Z = data.pivot_table(index=X_label, columns=Y_label, values=Z_label).values
    
    plt.figure()

    
    if log_scale_z:
        min_val = np.nanmin(Z[Z != -np.inf])
        if min_val == 0.0:
            min_val = 1.e-10
        
        max_val = np.nanmax(Z[Z != np.inf])
        if min_val < 0 :
            min_val_abs = np.nanmin(np.abs(Z))
            if min_val_abs == 0.0:
                min_val_abs = 1.e-9
            neg_array = np.logspace(np.log10(min_val_abs), np.log10(np.abs(min_val)), 13)
            pos_array = np.logspace(np.log10(min_val_abs), np.log10(max_val), 13)
            lvls = np.append(-np.flip(neg_array), pos_array)
        else:
            lvls = np.logspace(np.log10(min_val), np.log10(max_val), 25)
            
        l_f = LogFormatterSciNotation(base=10,labelOnlyBase=False)
        plt.contourf(X, Y, Z.T, norm=colors.SymLogNorm(linthresh = 1e-10, linscale = 1, 
                                                       vmin=min_val, vmax=max_val, base=10),levels=lvls,cmap='Spectral')
        plt.colorbar(ticks = SymmetricalLogLocator(subs=range(5),base=10,linthresh = 5e-8), format=l_f)
        
    else:
         plt.contourf(X, Y, Z.T,cmap='Spectral', alpha=0.8)
         plt.colorbar()
    plt.xlabel(X_label)
    plt.ylabel(Y_label)
    plt.title(Z_label)
    if log_scale_xy:
        plt.xscale('log')
        plt.yscale('log')
    
    return plt.gca()  

def plot_2D_line(data, X_label, Y_label, Z_label, param_label, param_val,
                        log_scale_xy = True, log_scale_z = True):
    
    ax = plot_2D(data, X_label, Y_label, Z_label, log_scale_xy, log_scale_z)
    
    X,Y = np.meshgrid(np.unique(data[X_label]),np.unique(data[Y_label]))
    Z = data.pivot_table(index=X_label, columns=Y_label, values=param_label).values
    
    CS = ax.contour(X, Y, Z.T, levels=param_val, colors='k')
    plt.clabel(CS, inline=True, fontsize=8)
    plt.show()
    return ax

def plot_2D_filter_line(data, X_label, Y_label, Z_label, param_filter_labels, param_filter_vals, threshold,
                        param_line_label, param_line_val, log_scale_xy = True, log_scale_z = True):
    
    ax = plot_2D_filter(data, X_label, Y_label, Z_label, param_filter_labels, param_filter_vals, threshold, log_scale_xy, log_scale_z)
    
    X,Y = np.meshgrid(np.unique(data[X_label]),np.unique(data[Y_label]))
    Z = data.pivot_table(index=X_label, columns=Y_label, values=param_line_label).values
    
    CS = ax.contour(X, Y, Z.T, levels=param_line_val, colors='k')
    plt.clabel(CS, inline=True, fontsize=8)
    plt.show()
    return ax

def plot_2D_filter_lines(data, X_label, Y_label, Z_label, param_filter_labels, param_filter_vals, threshold,
                        param_line_labels, param_line_vals, display_freq = True, log_scale_xy = True, log_scale_z = True):
    
    ax = plot_2D_filter(data, X_label, Y_label, Z_label, param_filter_labels, param_filter_vals, threshold, log_scale_xy, log_scale_z)
    for pindex, plabel in enumerate(param_line_labels):
        
        X,Y = np.meshgrid(np.unique(data[X_label]),np.unique(data[Y_label]))
        Z = data.pivot_table(index=X_label, columns=Y_label, values=plabel).values
        
        CS = ax.contour(X, Y, Z.T, levels=param_line_vals[pindex], colors='k')
        
        if display_freq :
            fmt = {}
            strs = [plabel]
            for l, s in zip(CS.levels, strs):
                fmt[l] = s
            plt.clabel(CS, inline=True, fontsize=12,fmt=fmt)
        else:
            plt.clabel(CS, inline=True, fontsize=8)
    plt.show()
    return ax

def plot_2D_filter(data, X_label, Y_label, Z_label, param_label, param_val, threshold, 
                   log_scale_xy = True, log_scale_z = True):
    
    filtered_data = data.copy()
    
    for i, param_l in enumerate(param_label):
        condition= np.abs(data[param_l] - param_val[i]) > threshold[i]
        filtered_data = filtered_data[ condition ]
    
    return plot_2D(filtered_data, X_label, Y_label, Z_label, log_scale_xy, log_scale_z)


def plot_2D_fit(model, data, X_label, Y_label, Z_label, param, log_scale_xy=True, log_scale_z = True):
    
    ax =  plot_2D(data, X_label, Y_label, Z_label, log_scale_xy, log_scale_z)
    X, Y = np.meshgrid(np.unique(data[X_label]), np.unique(data[Y_label]))
    Z = model(X, Y, param)

    CS = ax.contour(X, Y, Z, 10, colors='black')
    plt.clabel(CS, inline=True, fontsize=8)
    plt.show()
    return ax


def plot_1D(data, var1, var2, param_label=None, param_val=None, comp=None, threshold=None):
    filtered_data = data.copy()
    
    if param_label is not None and param_val is not None: 
        for i, param_l in enumerate(param_label):
            if comp[i] == '=':
                condition= np.abs(filtered_data[param_l] - param_val[i]) == threshold[i]
            if comp[i] == '<':
                condition= np.abs(filtered_data[param_l] - param_val[i]) < threshold[i]
            if comp[i] == '>':
                condition= np.abs(filtered_data[param_l] - param_val[i]) > threshold[i]
            filtered_data = filtered_data[ condition ]

    fig, ax = plt.subplots()
    filtered_data.plot(x=var1, y=var2, ax=ax)
    ax.set(xlabel=var1, ylabel=var2)
    plt.show()
    return ax

In [ ]:
plot_1D(dataset, 'EL/ECs', 'omega_01')
plot_2D(dataset, 'EL/EJ', 'EJ/ECj', 'omega_01', log_scale_xy = False, log_scale_z = True)
plot_2D(dataset, 'EL/EJ', 'EJ/ECj', 'T1_01', log_scale_xy = False, log_scale_z = True)
plot_2D(dataset, 'EL/EJ', 'EJ/EC', 'T2', log_scale_xy = False, log_scale_z = True)